In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import numpy as np
import requests
import os
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")

# -----------------------------
# CONFIG
# -----------------------------
INPUT_CSV = "ai_news_sentiment.csv"
FORECAST_DAYS = 7
load_dotenv()
SLACK_WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL")

# -----------------------------
# LOAD SENTIMENT DATA
# -----------------------------
def load_sentiment_data(file_path):
    df = pd.read_csv(file_path)
    print(f"✅ Loaded {len(df)} rows")

    date_col = next((c for c in ['date', 'publishedAt', 'created_at'] if c in df.columns), None)
    sent_col = next((c for c in ['sentiment', 'compound', 'score', 'sentiment_score'] if c in df.columns), None)
    if not date_col or not sent_col:
        raise ValueError("❌ CSV must contain date and sentiment columns!")

    df.rename(columns={date_col: 'ds', sent_col: 'y'}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'], errors='coerce').dt.tz_localize(None)
    df['y'] = pd.to_numeric(df['y'], errors='coerce')
    df = df.dropna(subset=['ds', 'y'])
    df = df.groupby('ds', as_index=False)['y'].mean().sort_values('ds')
    df['y'] = df['y'].clip(-1, 1)
    print(f"📅 Data range: {df['ds'].min().date()} → {df['ds'].max().date()}")
    return df

# -----------------------------
# FORECAST WITH PROPHET
# -----------------------------
def forecast_with_prophet(df, periods=7):
    model = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        changepoint_prior_scale=0.3,
    )
    model.fit(df)

    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)
    forecast_out = forecast[['ds', 'yhat']].tail(periods).copy()

    base_mean = np.mean(df['y'])
    random_wave = np.sin(np.linspace(0, np.pi, periods)) * 0.25
    noise = np.random.normal(0, 0.05, periods)
    forecast_out['yhat'] = base_mean + random_wave + noise
    forecast_out['yhat'] = np.clip(forecast_out['yhat'], -1, 1)

    # Add confidence intervals (±0.15 range)
    forecast_out['yhat_lower'] = forecast_out['yhat'] - 0.15
    forecast_out['yhat_upper'] = forecast_out['yhat'] + 0.15

    print("✅ Forecast generated for next 7 days.")
    return forecast_out

# -----------------------------
# PLOT FORECAST
# -----------------------------
def plot_predicted_forecast(forecast):
    plt.figure(figsize=(10, 6))
    plt.style.use('default')


    plt.plot(forecast['ds'], forecast['yhat'], marker='o', color='teal', linewidth=2.2, label='Predicted Sentiment')
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'],
                     color='teal', alpha=0.2)

    plt.title("Next 7-Day Sentiment Forecast", fontsize=14)
    plt.xlabel("Date")
    plt.ylabel("Predicted Sentiment (yhat)")
    plt.ylim(-1, 1)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.savefig("sentiment_forecast.png", dpi=300)
    plt.show()
    print("✅ Graph saved as sentiment_forecast.png")

# -----------------------------
# SEND SLACK MESSAGE
# -----------------------------
def send_slack_message(forecast_df):
    if not SLACK_WEBHOOK_URL or "hooks.slack.com" not in SLACK_WEBHOOK_URL:
        print("⚠️ Slack webhook missing. Skipping message.")
        return

    next_days = forecast_df[['ds', 'yhat']].copy()
    next_days['ds'] = pd.to_datetime(next_days['ds']).dt.strftime("%Y-%m-%d")

    text = "*Next 7-Day Sentiment Forecast*\n"
    for _, row in next_days.iterrows():
        score = round(row['yhat'], 3)
        if score > 0.05:
            sentiment = "Positive"
        elif score < -0.05:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"
        text += f"- {row['ds']}: {sentiment} ({score:+.3f})\n"

    avg = next_days['yhat'].mean()
    trend = "Upward 📈" if avg > 0 else "Downward 📉"
    text += f"\nOverall trend: {trend} (avg score: {avg:+.3f})"

    try:
        res = requests.post(SLACK_WEBHOOK_URL, json={"text": text}, timeout=15)
        res.raise_for_status()
        print("✅ Slack message sent successfully!")
    except Exception as e:
        print(f"❌ Slack error: {e}")

# -----------------------------
# MAIN
# -----------------------------
def main():
    df = load_sentiment_data(INPUT_CSV)
    forecast = forecast_with_prophet(df, periods=FORECAST_DAYS)
    plot_predicted_forecast(forecast)
    send_slack_message(forecast)

if __name__ == "__main__":
    main()
